# Notebook Setup

In [1]:
if 'google.colab' in str(get_ipython()):
  IN_COLLAB = True
else:
  IN_COLLAB = False

#TODO: CHANGE THIS BASED ON YOUR OWN LOCAL SETTINGS
MY_HOME_ABS_PATH = "/content/drive/MyDrive/W210/co2-flux-hourly-gpp-modeling"

In [2]:
if IN_COLLAB:
  from google.colab import drive
  drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


## Import Modules

In [3]:
# install required modules quietly
required_packages = ['geopandas', 'pyspark', 'azure-storage-blob']

for p in required_packages: 
  try:
      __import__(p)
  except ImportError:
      %pip install {p} --quiet

In [4]:
import os
os.environ["PYARROW_IGNORE_TIMEZONE"] = "1"
import math
import json

import pyspark.pandas as pd
from calendar import monthrange
from datetime import datetime
from io import BytesIO

import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

# Load locale custome modules
import sys
if IN_COLLAB:
  os.chdir(MY_HOME_ABS_PATH)
  sys.path.insert(0,os.path.abspath("./code/src/tools"))
else:
  sys.path.append(os.path.abspath("./code/src/tools"))

from CloudIO.AzStorageClient import AzStorageClient
from data_pipeline_lib import *

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [5]:
# Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Check Spark Session Information
spark

## Define Local Files System Constants

In [6]:
root_dir =  MY_HOME_ABS_PATH
tmp_dir =  root_dir + os.sep + '.tmp'
raw_data_dir = tmp_dir
data_dir = root_dir + os.sep + 'data'
cred_dir = root_dir + os.sep + '.cred'
az_cred_file = cred_dir + os.sep + 'azblobcred.json'

if IN_COLLAB:
  raw_data_dir = "/content/drive/MyDrive/CO2_flux_gpp_modeling/DS_capstone_23Spring_CO2/Data/half_hourly_data"

site_metadata_filename = data_dir + os.sep + 'site-metadata.csv'

# Output
tag = "0"
container = "test-sites-data"
blob_name = f"test-sites-data_v_{tag}.parquet" #Advisor suggested features only

In [7]:
# Selected Test Sites
test_sites = ["US-GLE", # ENF, Cold
              "US-AR1", # GRA, Temperate
              "US-Seg", # GRA, Arid
              "US-FR2", # WSA, Temperate
              "ES-LM2", # WSA, Arid
              "CA-Cbo", # DBF, Cold
              "FR-Lam", # CRO, Temperate
              "IT-Cpz", # EBF, Temperate
              "CN-Cha", # MF Cold
              "IT-Lsn", # OSH, Temperate
              ]

# Get Selected Site Metadata

In [8]:
# Load site metadata
included_site_features = ['site_id', 'filename', 'elevation', 'lat', 'long',
                          'koppen_sub', 'koppen_main', 'koppen_name', 'koppen_main_name',
                          'c3c4', 'c4_percent']
site_metadata_df = pd.read_csv(site_metadata_filename, usecols = included_site_features)

# only focus on target sites
site_metadata_df= site_metadata_df.loc[site_metadata_df['site_id'].isin(test_sites)]
print(f"size:{site_metadata_df.shape}")
site_metadata_df.reset_index(inplace=True, drop=True)
site_metadata_df

size:(10, 11)


,site_id,elevation,lat,long,koppen_sub,koppen_main,koppen_name,koppen_main_name,c3c4,c4_percent,filename
0,CN-Cha,NaN,42.40250,128.09580,22,4,Dwb,Cold,C3,12.17000,data_full_half_hourly_raw_v0_1_CN-Cha.csv
1,IT-Cpz,68.00000,41.70525,12.37611,8,3,Csa,Temperate,C3,0.00000,data_full_half_hourly_raw_v0_1_IT-Cpz.csv
2,US-GLE,3197.00000,41.36653,-106.23990,27,4,Dfc,Cold,C3,0.16000,data_full_half_hourly_raw_v0_1_US-GLE.csv
3,CA-Cbo,120.00000,44.31670,-79.93330,26,4,Dfb,Cold,C3,1.09000,data_full_half_hourly_raw_v0_1_CA-Cbo.csv
4,US-AR1,611.00000,36.42670,-99.42000,14,3,Cfa,Temperate,C4,27.54000,data_full_half_hourly_raw_v0_1_US-AR1.csv
5,US-FR2,271.90000,29.94950,-97.99620,14,3,Cfa,Temperate,C3,18.37000,data_full_half_hourly_raw_v0_1_US-FR2.csv
6,US-Seg,1622.00000,34.36230,-106.70190,5,2,BWk,Arid,mix,0.24000,data_full_half_hourly_raw_v0_1_US-Seg.csv
7,ES-LM2,270.00000,39.93459,-5.77588,7,2,BSk,Arid,C3,1.30000,data_full_half_hourly_raw_v0_1_ES-LM2.csv
8,FR-Lam,180.00000,43.49644,1.23788,14,3,Cfa,Temperate,rotation,3.10000,data_full_half_hourly_raw_v0_1_FR-Lam.csv
9,IT-Lsn,1.00000,45.74048,12.75030,14,3,Cfa,Temperate,C3,2.70000,data_full_half_hourly_raw_v0_1_IT-Lsn.csv


# Run Data Pipeline on Gold Sample Sites

In [9]:
all_features = ['TIMESTAMP_START', 'TIMESTAMP_END', 'TA_F', 'TA_F_QC', 'TA_ERA',
       'SW_IN_POT', 'SW_IN_F', 'SW_IN_F_QC', 'SW_IN_ERA', 'LW_IN_F',
       'LW_IN_F_QC', 'LW_IN_ERA', 'VPD_F', 'VPD_F_QC', 'VPD_ERA', 'P_F',
       'P_F_QC', 'P_ERA', 'PA_F', 'PA_F_QC', 'PA_ERA', 'NETRAD', 'PPFD_IN',
       'G_F_MDS', 'G_F_MDS_QC', 'LE_F_MDS', 'LE_F_MDS_QC', 'LE_CORR',
       'H_F_MDS', 'H_F_MDS_QC', 'H_CORR', 'NEE_VUT_REF', 'NEE_VUT_REF_QC',
       'NEE_CUT_REF', 'NEE_CUT_REF_QC', 'GPP_NT_VUT_REF', 'GPP_DT_VUT_REF',
       'GPP_NT_CUT_REF', 'GPP_DT_CUT_REF', 'RECO_NT_VUT_REF',
       'RECO_DT_VUT_REF', 'RECO_NT_CUT_REF', 'RECO_DT_CUT_REF', 'datetime',
       'year', 'month', 'day', 'hour', 'SITE_ID', 'date', 'NEE_VUT_REF_qa',
       'SW_DIF', 'EVI', 'NDVI', 'NIRv', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6',
       'b7', 'IGBP', 'koppen']

In [ ]:
# Define features and target variables of the data pipelines
included_features = ['TA_ERA', 'SW_IN_ERA', 'LW_IN_ERA', 'VPD_ERA', 'P_ERA', 'PA_ERA',
                     'datetime', 'year', 'month', 'day', 'hour', 'date',
                     'EVI', 'NDVI', 'NIRv', 'b1', 'b2', 'b3', 'b4', 'b5', 'b6', 'b7', 
                     'IGBP', 'koppen']
target_variable_qc = ['NEE_VUT_REF_QC']
target_variable = ['GPP_NT_VUT_REF']

In [10]:
# Get Train Dataset

# initial data clean and feature selections from raw data
data_df = data_cleanup(raw_data_dir, site_metadata_df[['site_id','filename']],
                  target_variable, target_variable_qc,
                  included_features)
print(f"Data size after cleanup: {data_df.shape}")

# Merge with site metadata
data_df = merge_site_metadata(data_df, site_metadata_df.drop(['filename'], axis=1))
print(f"Data size after after merged with site metadata: {data_df.shape}")

# Drop rows with NA
check_and_drop_na(data_df)
print(f"Data size after after final drop: {data_df.shape}")

display(data_df.head())


CN-Cha: (16228, 27)
IT-Cpz: (59175, 27)
US-GLE: (54687, 27)
CA-Cbo: (79273, 27)
US-AR1: (28956, 27)
US-FR2: (30426, 27)
US-Seg: (91884, 27)
ES-LM2: (58806, 27)
FR-Lam: (115812, 27)
IT-Lsn: (40182, 27)
Data size after cleanup: (575429, 27)
Data size after after merged with site metadata: (575429, 36)
Data has NA.


,TA_ERA,SW_IN_ERA,LW_IN_ERA,VPD_ERA,P_ERA,PA_ERA,GPP_NT_VUT_REF,datetime,year,month,day,hour,date,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen,minute,site_id,elevation,lat,long,koppen_sub,koppen_main,koppen_name,koppen_main_name,c3c4,c4_percent
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,16228,0,0,0,0,0,0,0,0


Data size after after final drop: (559201, 36)


,TA_ERA,SW_IN_ERA,LW_IN_ERA,VPD_ERA,P_ERA,PA_ERA,GPP_NT_VUT_REF,datetime,year,month,day,hour,date,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,IGBP,koppen,minute,site_id,elevation,lat,long,koppen_sub,koppen_main,koppen_name,koppen_main_name,c3c4,c4_percent
16228,4.66000,5.44900,218.92000,2.90400,0.00000,100.76500,2.19090,2001-01-01 07:30:00,2001,1,1,7,2001-01-01,0.30272,0.74517,0.12452,0.02440,0.16710,0.01800,0.02910,0.23520,0.09770,0.04840,EBF,Temperate,30,IT-Cpz,68.00000,41.70525,12.37611,8,3,Csa,Temperate,C3,0.00000
16229,5.08600,71.31800,218.92000,2.95200,0.00000,100.78900,2.83647,2001-01-01 08:00:00,2001,1,1,8,2001-01-01,0.30272,0.74517,0.12452,0.02440,0.16710,0.01800,0.02910,0.23520,0.09770,0.04840,EBF,Temperate,0,IT-Cpz,68.00000,41.70525,12.37611,8,3,Csa,Temperate,C3,0.00000
16230,5.51100,132.32000,218.92000,3.00000,0.00000,100.81300,4.03906,2001-01-01 08:30:00,2001,1,1,8,2001-01-01,0.30272,0.74517,0.12452,0.02440,0.16710,0.01800,0.02910,0.23520,0.09770,0.04840,EBF,Temperate,30,IT-Cpz,68.00000,41.70525,12.37611,8,3,Csa,Temperate,C3,0.00000
16231,5.93700,187.41100,218.92000,3.04800,0.00000,100.83700,7.79772,2001-01-01 09:00:00,2001,1,1,9,2001-01-01,0.30272,0.74517,0.12452,0.02440,0.16710,0.01800,0.02910,0.23520,0.09770,0.04840,EBF,Temperate,0,IT-Cpz,68.00000,41.70525,12.37611,8,3,Csa,Temperate,C3,0.00000
16232,6.36200,235.65000,218.92000,3.09600,0.00000,100.86100,6.93281,2001-01-01 09:30:00,2001,1,1,9,2001-01-01,0.30272,0.74517,0.12452,0.02440,0.16710,0.01800,0.02910,0.23520,0.09770,0.04840,EBF,Temperate,30,IT-Cpz,68.00000,41.70525,12.37611,8,3,Csa,Temperate,C3,0.00000


In [11]:
data_df.describe()

,TA_ERA,SW_IN_ERA,LW_IN_ERA,VPD_ERA,P_ERA,PA_ERA,GPP_NT_VUT_REF,year,month,day,hour,EVI,NDVI,NIRv,b1,b2,b3,b4,b5,b6,b7,minute,elevation,lat,long,koppen_sub,koppen_main,c4_percent
count,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000,559201.00000
mean,15.99888,372.78984,328.34315,10.09542,0.04491,93.80726,5.89935,2011.04924,6.58824,15.76970,12.00068,0.29945,0.49832,0.13357,0.09872,0.26706,0.05637,0.08935,0.28554,0.22718,0.14550,15.00486,715.54907,40.39754,-47.85888,14.12260,2.97008,3.60790
std,9.44850,263.79306,53.34800,9.59278,0.21359,9.93809,7.84509,4.85439,3.04618,8.81670,3.70952,0.13835,0.22863,0.07576,0.07196,0.06416,0.05761,0.05835,0.06413,0.09469,0.09622,15.00001,979.83832,4.38791,51.53537,7.69401,0.71284,6.89128
min,-30.68700,0.00100,107.63900,0.00000,0.00000,66.03100,-49.38330,2001.00000,1.00000,1.00000,4.00000,-0.09768,-0.02273,-0.01764,0.01480,0.09860,0.00500,0.01930,0.09130,0.02990,0.01230,0.00000,1.00000,29.94950,-106.70190,5.00000,2.00000,0.00000
25%,9.80400,137.52300,289.93600,3.35200,0.00000,84.79300,0.42672,2007.00000,4.00000,8.00000,9.00000,0.19767,0.32305,0.07163,0.04730,0.21550,0.02510,0.05220,0.24010,0.15750,0.07070,0.00000,120.00000,36.42670,-106.23990,7.00000,2.00000,0.24000
50%,16.40000,340.39600,334.09000,6.99500,0.00000,98.73900,2.93147,2011.00000,7.00000,16.00000,12.00000,0.29334,0.52381,0.12294,0.07760,0.26630,0.03980,0.07450,0.29010,0.21110,0.11770,30.00000,180.00000,41.70525,-79.93330,14.00000,3.00000,1.09000
75%,22.74500,580.10600,369.34900,13.61200,0.00000,99.85900,9.38355,2015.00000,9.00000,23.00000,15.00000,0.38945,0.68621,0.18062,0.13040,0.31170,0.06460,0.10430,0.33200,0.29520,0.19270,30.00000,1622.00000,43.49644,1.23788,14.00000,3.00000,3.10000
max,44.08700,1050.62600,473.08500,80.09100,13.03500,103.92100,83.61300,2020.00000,12.00000,31.00000,23.00000,1.41056,0.90239,0.38458,0.81200,0.77620,0.76400,0.80530,0.44180,0.47920,0.43470,30.00000,3197.00000,45.74048,12.75030,27.00000,4.00000,27.54000


In [12]:
data_df.site_id.unique()

array(['IT-Cpz', 'US-GLE', 'CA-Cbo', 'US-AR1', 'US-FR2', 'US-Seg',
       'ES-LM2', 'FR-Lam', 'IT-Lsn'], dtype=object)

# Upload Data to Azure Storage Blob as Parquet
**Run with Caution!!!**

In [13]:
# Upload to Azure Storage Blob
# ref: https://stackoverflow.com/a/54666079
parquet_file = BytesIO()
data_df.to_parquet(parquet_file, engine='pyarrow')
parquet_file.seek(0)

azStorageClient = AzStorageClient(az_cred_file)
azStorageClient.uploadBlob(container, blob_name, parquet_file, overwrite=True)

File uploaded to test-sites-data/test-sites-data_v_0.parquet
